In [1]:
import xarray as xr
from dask.distributed import Client, progress
import zarr

In [2]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=10)
cluster

In [3]:
client = Client(cluster)
client

Client Scheduler: tcp://192.168.19.194:37433 Dashboard: /user/aimeeb/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [28]:
import os

chunks = {'x': 1000, 'y': 1000}

base_path = 'data/snowdepth/netcdf'

concat_dim = []
files = []
for file in os.listdir(base_path):
    if file.endswith('int.nc'):
        concat_dim.append(file.split('_')[0])
        files.append(f"{base_path}/{file}")

print(files)
print(concat_dim)
ds = xr.open_mfdataset(files, chunks=chunks, concat_dim=concat_dim, parallel=True)
ds

['data/snowdepth/netcdf/2017/20170401_SUPERsnow_depth_3m_int.nc', 'data/snowdepth/netcdf/2017/20170502_SUPERsnow_depth_3m_int.nc', 'data/snowdepth/netcdf/2017/20170816_SUPERsnow_depth_3m_int.nc', 'data/snowdepth/netcdf/2017/20170717_SUPERsnow_depth_3m_int.nc', 'data/snowdepth/netcdf/2017/20170303_SUPERsnow_depth_3m_int.nc', 'data/snowdepth/netcdf/2017/20170604_SUPERsnow_depth_3m_int.nc', 'data/snowdepth/netcdf/2017/20170727_SUPERsnow_depth_3m_int.nc', 'data/snowdepth/netcdf/2017/20170709_SUPERsnow_depth_3m_int.nc', 'data/snowdepth/netcdf/2017/20170129_SUPERsnow_depth_3m_int.nc']
['20170401', '20170502', '20170816', '20170717', '20170303', '20170604', '20170727', '20170709', '20170129']


/srv/conda/envs/notebook/lib/python3.7/site-packages/xarray/backends/api.py:783: FutureWarning: In xarray version 0.13 `auto_combine` will be deprecated.
  coords=coords)
/srv/conda/envs/notebook/lib/python3.7/site-packages/xarray/backends/api.py:783: FutureWarning: Also `open_mfdataset` will no longer accept a `concat_dim` argument.
To get equivalent behaviour from now on please use the new
`combine_nested` function instead (or the `combine='nested'` option to
`open_mfdataset`).The datasets supplied do not have global dimension coordinates. In
future, to continue concatenating without supplying dimension
coordinates, please use the new `combine_nested` function (or the
`combine='nested'` option to open_mfdataset.
  coords=coords)


<xarray.Dataset>
Dimensions:              (concat_dim: 9, x: 17569, y: 17002)
Coordinates:
  * x                    (x) float64 2.54e+05 2.54e+05 ... 3.067e+05 3.067e+05
  * y                    (y) float64 4.179e+06 4.179e+06 ... 4.23e+06 4.23e+06
  * concat_dim           (concat_dim) <U8 '20170401' '20170502' ... '20170129'
Data variables:
    transverse_mercator  (concat_dim) |S1 b'' b'' b'' b'' b'' b'' b'' b'' b''
    Band1                (concat_dim, y, x) float32 dask.array<shape=(9, 17002, 17569), chunksize=(1, 1000, 1000)>
Attributes:
    GDAL_AREA_OR_POINT:  Area
    Conventions:         CF-1.5
    GDAL:                GDAL 2.4.1, released 2019/03/15
    history:             Wed Sep 11 15:46:45 2019: GDAL CreateCopy( /srv/shar...

In [29]:
# Write zarr to local storage
compressor = zarr.Blosc(cname='zstd', clevel=5, shuffle=zarr.Blosc.AUTOSHUFFLE)
encoding = {v: {'compressor': compressor} for v in ds.data_vars}
ds.to_zarr(store='snow.zarr', mode='w', encoding=encoding)

In [49]:
zarr_store = xr.open_zarr('snow.zarr')

In [55]:
xmean = zarr_store.mean(dim='concat_dim')

In [57]:
%%time
values = xmean['Band1'].values
print(values)

[[  0.        0.        0.      ...   0.        0.        0.     ]
 [  0.        0.        0.      ...   0.        0.        0.     ]
 [  0.        0.        0.      ...   0.        0.        0.     ]
 ...
 [188.11111 205.77777 220.11111 ...   0.        0.        0.     ]
 [200.11111 209.77777 230.66667 ...   0.        0.        0.     ]
 [155.88889 171.55556 212.      ...   0.        0.        0.     ]]
CPU times: user 8.77 s, sys: 1.79 s, total: 10.6 s
Wall time: 27.7 s
